# 랭체인으로 RAG구현하기 소스코드 1

In [ ]:
!pip install langchain openai  # 필요한 라이브러리 설치

import os
from google.colab import files
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


In [ ]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = "your-openai-api-key"

# 1. 파일 업로드
uploaded = files.upload()

# 2. 업로드된 파일 이름 가져오기 (업로드된 파일을 바로 사용)
uploaded_filename = list(uploaded.keys())[0]

# 3. 파일 로더 설정
data_loader = UnstructuredFileLoader(uploaded_filename)

In [ ]:
# 4. 텍스트 분할 설정
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=500,
    chunk_overlap=50
)

# 5. 텍스트 로드 및 분할
docs = data_loader.load_and_split(text_splitter=splitter)

# 6. 임베딩 설정 및 캐싱
embeddings = OpenAIEmbeddings()  # OpenAI 임베딩 인스턴스 생성

In [ ]:
# 캐시 디렉토리 설정 (Google Colab 작업 디렉토리에 생성됨)
cache_dir = LocalFileStore("./.cache/")

# 캐시 지원 임베딩 인스턴스 생성
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# 7. 벡터 스토어 생성
vectorstore = Chroma.from_documents(docs, cached_embeddings)

# 8. 검색기 인스턴스 생성
retriever = vectorstore.as_retriever()

# 9. LLM 인스턴스 생성
model = ChatOpenAI()

# 10. RetrievalQA 체인 생성
chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type="map_reduce",
    retriever=retriever,
)

In [ ]:
# 11. 질문 실행
answer = chain.run("What is the capital of France?")
print(answer)  # "Paris"와 같은 답변이 출력됩니다.


In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chatgpt = ChatOpenAI(model_name="gpt-3.5-turbo", streaming=True, callbacks=[StreamingStdOutCallbackHandler()], temperature = 1)
answer = chatgpt.predict("왜 파이썬이 가장 인기있는 프로그래밍 언어야?")

#실시간 타이핑이 보이도록

# 랭체인을 사용하여 RAG구현 소스코드 2

Langchain을 활용한 RAG 구현 소스코드 2


In [ ]:
!pip install langchain unstructured pypdf pdf2image docx2txt pdfminer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 70.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 56.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.8 MB/s eta 0:00:00
  Created wheel for docx2txt: filename=docx2txt-0.8-py3-none-any.whl siz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com")
data = loader.load()
print(data[0].page_content)

In [ ]:
from langchain.document_loaders import UnstructuredURLLoader

urls = [
    "page"
    "page"
]
loader = UnstructuredURLLoader(urls=urls)
data

# PDF다운로더

PDF Document Loader

In [ ]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.0 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/drive/MyDrive/R152r2E.pdf")
pages = loader.load_and_split()

ValueError: File path /content/drive/MyDrive/R152r2E.pdf is not a valid file or url

In [ ]:
print(pages[0].page_content)

GE.23 -11213(E)  
  Agreement  
  Concerning the  Adoption of Harmonized Technical United Nations 
Regulations for Wheeled Vehicles, Equipment and Parts which can be 
Fitted and/or be Used on Wheeled Vehicles and the Conditions for 
Reciprocal Recognition of Approvals Granted on the Basis of these 
United Nat ions Regulations * 
  (Revision 3, including the amendments which entered into force on 14 September 2017)  
_________  
  Addendum 1 51 – UN Regulation No. 1 52 
  Revision 2 
Incorporating all valid text up to:  
Supplement 1 to the 01 series of amendments – Date of entry into force: 3 January 2021  
Supplement 2 to the 01 series of amendments – Date of entry into force: 30 September 2021  
02 series of amendments – Date of entry into force: 30 September 2021  
  Uniform  provisions concerning the approval of motor vehicles with 
regard to the Advanced Emergency Braking System (AEBS) for M 1 and 
N1 vehicles  
_________  
UNITED NATIONS  
  
 * Former titles of the Agreement:  


In [ ]:
len(pages)

36

In [ ]:
print(pages[1].page_content)

E/ECE/TRANS/505/Rev.3/Add.1 51/Rev. 2 
2 This document is meant purely as documentation tool. The authentic and legal binding text s 
are: ECE/TRANS/WP.29/2020/69 , ECE/TRANS/WP.29/2021/16  and 
ECE/TRANS/WP.29/2021/18 .


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len
)

In [ ]:
pages = loader.load_and_split()



In [ ]:
texts = text_splitter.split_documents(pages)
len(texts)

122

In [ ]:
print(texts[1].page_content)

regard to the Advanced Emergency Braking System (AEBS) for M 1 and 
N1 vehicles  
_________  
UNITED NATIONS  
  
 * Former titles of the Agreement:  
  Agreement concerning the Adoption of Uniform Conditions of Approval and Reciprocal Recognition of 
Approval for Motor Vehicle Equipment and Parts, done at Geneva on 20 March 1958 (original version);  
  Agreement concerning the Adop tion of Uniform Technical Prescriptions for Wheeled Vehicles, 
Equipment and Parts which can be Fitted and/or be Used on Wheeled Vehicles and the Conditions for 
Reciprocal Recognition of Approvals Granted on the Basis of these Prescriptions, done at Geneva on 
5 October 1995 (Revision 2).   E/ECE/TRANS/505/Rev.3/Add.1 51/Rev. 2 
   
 
15 June 2023


In [ ]:
char_list = []
for i in range(len(texts)):
  char_list.append(len(texts[i].page_content))
print(char_list)

[997, 736, 220, 879, 909, 856, 695, 994, 996, 235, 964, 952, 977, 375, 928, 953, 967, 995, 216, 980, 993, 910, 598, 934, 961, 993, 583, 994, 997, 435, 923, 966, 563, 992, 978, 219, 982, 977, 848, 993, 992, 459, 959, 944, 912, 944, 419, 933, 916, 988, 477, 995, 995, 956, 836, 926, 945, 943, 260, 976, 973, 994, 651, 943, 968, 983, 781, 918, 965, 959, 645, 969, 944, 983, 462, 969, 996, 385, 941, 973, 989, 452, 681, 984, 934, 994, 569, 919, 949, 935, 996, 197, 977, 942, 953, 882, 997, 962, 990, 974, 322, 984, 966, 981, 704, 969, 892, 976, 917, 307, 701, 968, 995, 684, 912, 975, 465, 990, 784, 969, 894, 520]


# 텍스트 임베딩

**TEXT Embedding**

In [ ]:
!pip install openai langchain pypdf tiktoken

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(openai_api_key = "")

In [ ]:
embeddings = embeddings_model.embed_documents(
    [
        "Hello world",
        "How are you?",
        "제 이름은 홍길동입니다."
        "공부가 너무 싫어요"
        "랭체인이 뭔지 아시나요?"
    ]
)
len(embeddings),len(embeddings[0])

In [ ]:
embedded_query_q = embeddings_model.embed_quary("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = embeddings_model.embed_quary("이 대화에서 언급된 이름은 홍길동입니다.")
print(len(embedded_query_q),len(embedded_query_a))

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A,B):
  return dot(A,B)/(norm(A)*norm(B))

In [ ]:
print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_q, embeddings[1]))
print(cos_sim(embedded_query_q, embeddings[3]))
#임베딩 모델간의 유사도 확인

허깅페이스 임베딩

# 허깅페이스 모델을 사용한 임베딩

In [ ]:
!pip install sentence_transformers

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

ModuleNotFoundError: Module langchain_community.embeddings not found. Please install langchain-community to access this module. You can install it using `pip install -U langchain-community`

In [ ]:
embeddings = hf.embed_documents(
    [
    "today is monday",
    "weather is nice today",
    "what's the problem?",
    "langhcain in useful",
    "Hello World!",
    "my name is morris"
    ]
)

In [ ]:
BGE_query_q = hf.embed_query("Hello? who is this?")
BGE_query_a = hf.embed_query("hi this is harrison")

print(cos_sim(BGE_query_q, BGE_query_a))
print(cos_sim(BGE_query_q, embeddings[1]))
print(cos_sim(BGE_query_q, embeddings[5]))

In [ ]:
sentences = [
    "안녕하세요",
    "제 이름은 홍길동입니다.",
    "이름이 무엇인가요?",
    "랭체인은 유용합니다.",
    "홍길동 아버지의 이름은 홍상직입니다."
    ]
ko_embeddings = hf.embed_documents(sentences)

In [ ]:
BGE_query_q_2 = hf.embed_query("홍길동은 아버지를 아버지라 부르지 못하였습니다. 홍길동 아버지의 이름은 무엇입니까?")
BGE_query_a_2 = hf.embed_query("홍길동의 아버지는 엄했습니다.")


print("질문: 홍길동은 아버지를 아버지라 부르지 못하였습니다. 홍길동 아버지의 이름은 무엇입니까? \n", "-"*100)
print("홍길동의 아버지는 엄했습니다. \t\t 문장 유사도: ", round(cos_sim(BGE_query_q_2, BGE_query_a_2),2))
print(sentences[1] + "\t\t\t 문장 유사도: ", round(cos_sim(BGE_query_q_2, ko_embeddings[1]),2))
print(sentences[3] + "\t\t\t 문장 유사도: ", round(cos_sim(BGE_query_q_2, ko_embeddings[3]),2))
print(sentences[4] + "\t 문장 유사도: ", round(cos_sim(BGE_query_q_2, ko_embeddings[4]),2))

# 벡터스토어를 이용하여 문서 임베딩 및 유사성 검색


벡터스토어

In [ ]:
!pip install chromadb tiktoken transformers sentence_transformers openai langchain pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader


# load the document and split it into chunks
loader = PyPDFLoader("/content/drive/MyDrive/R152r2E.pdf")
pages = loader.load_and_split()

# split it into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0, length_function = tiktoken_len)
docs = text_splitter.split_documents(pages)

# create the open-source embedding function
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "Alibaba-NLP/gte-Qwen2-7B-instruct"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/284 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/145k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

...

model-00004-of-00007.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00005-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00006-of-00007.safetensors:   0%|          | 0.00/3.66G [00:00<?, ?B/s]

model-00007-of-00007.safetensors:   0%|          | 0.00/2.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# load it into Chroma
db = Chroma.from_documents(docs, hf)

# query it
query = "(AEBS) is used in what situations?"
docs = db.similarity_search(query)

# print results
print(docs[0].page_content)

InvalidDimensionException: Embedding dimension 384 does not match collection dimensionality 768

In [ ]:
tiktoken_len(docs[0].page_content)

343

In [ ]:
# save to disk
db2 = Chroma.from_documents(docs, hf, persist_directory="./chroma_db")
docs = db2.similarity_search(query)

In [ ]:
# load from disk
db3 = Chroma(persist_directory="./chroma_db", embedding_function=hf)
docs = db3.similarity_search(query)
print(docs[0].page_content)

In [ ]:
docs = db3.similarity_search_with_relevance_scores(query, k=3)


print("가장 유사한 문서:\n\n {}\n\n".format(docs[0][0].page_content))
print("문서 유사도:\n {}".format(docs[0][1]))

# huggingface, chromaDB, Langchain을 사용한 RAG

In [ ]:
!pip install chromadb tiktoken transformers sentence_transformers openai langchain pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import openai
os.environ["OPENAI_API_KEY"] = ''

In [ ]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [ ]:
!pip install -U langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.4 MB/s eta 0:00:00


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/UNECEmergeData.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function = tiktoken_len)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

docsearch = Chroma.from_documents(texts, hf)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

openai = ChatOpenAI(model_name="gpt-4o-mini-2024-07-18",
                    streaming=True, callbacks=[StreamingStdOutCallbackHandler()],
                    temperature = 0)

qa = RetrievalQA.from_chain_type(llm = openai,
                                 chain_type = "stuff",
                                 retriever = docsearch.as_retriever(
                                    search_type="mmr",
                                    search_kwargs={'k':3, 'fetch_k': 10}),
                                 return_source_documents = True)

query = "What are the specific requirements and procedures outlined in the braking test conditions for a vehicle seeking approval, including the initial brake temperature range, test speed criteria for different vehicle categories, brake actuation force limits, and the necessary steps to be taken before and during the test to ensure compliance with the approval regulations?"
result = qa(query)

The specific requirements and procedures outlined in the braking test conditions for a vehicle seeking approval are as follows:

1. **Initial Brake Temperature**: The brakes must be at an initial temperature of at least 55 °C and not exceed 100 °C before the test begins.

2. **Test Speed Criteria**:
   - For general brake testing, the speed should be 100 km/h or 0.9 times the maximum speed (Vmax) of the vehicle, whichever is lower.
   - For high-speed testing applicable to vehicle categories L3, L4, L5, and L7 with a Vmax greater than 125 km/h, the test speed should be 0.8 Vmax.

3. **Brake Actuation Force Limits**:
   - Hand control: Up to 250 N.
   - Foot control: Up to 400 N for vehicle categories L3 and L4, and up to 500 N for vehicle categories L5 and L7.

4. **Test Procedure Steps**:
   - **Acceleration and Braking**: For each stop, the vehicle must be accelerated to the specified test speed and then the brakes should be actuated under the specified conditions.
   - **Brake Appli

In [ ]:
result

{'query': 'What are the specific conditions under which the ABS function must be disabled or disconnected according to the regulations mentioned in the provided context?',
 'result': 'According to the regulations mentioned in the provided context, the specific conditions under which the ABS function must be disabled or disconnected are between 40 km/h and 20 km/h, when the vehicle is lightly loaded, and when the engine is disconnected.',
 'source_documents': [Document(metadata={'page': 40, 'source': '/content/drive/MyDrive/UNECEmergeData.pdf'}, page_content='(b) The anti -lock system shall be either disconnected or inoperative (ABS \nfunction disabled), between 40 km/h and 20 km/h.  \n(c) Lightly loaded.  \n(d) Engine disconnected.  \n1.3. Test conditions and procedure:  \n(a) Initial brake temperature:   ≥ 55 °C and  ≤ 100 °C. \n(b) Test speed: 60 km/h or 0.9 Vmax, whichever is lower.'),
  Document(metadata={'page': 55, 'source': '/content/drive/MyDrive/UNECEmergeData.pdf'}, page_cont